In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import pandas as pd

# Load the training and validation data
train_df = pd.read_csv("train-data.tsv", sep="\t", header=None, names=["label", "message"])
valid_df = pd.read_csv("valid-data.tsv", sep="\t", header=None, names=["label", "message"])

# Display the first few rows
print(train_df.head())
print(valid_df.head())


In [ ]:
train_df["label"] = train_df["label"].map({"ham": 0, "spam": 1})
valid_df["label"] = valid_df["label"].map({"ham": 0, "spam": 1})


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer # Import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Define max_length
max_length = 100 # or any suitable value

# Create and fit tokenizer on training data
tokenizer = Tokenizer() # Create a Tokenizer object
tokenizer.fit_on_texts(train_df['message'])

# Define vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Create the model
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_length)) # Increased embedding dimension
model.add(LSTM(64)) # Increased LSTM units
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Prepare training data
training_sequences = tokenizer.texts_to_sequences(train_df['message'])
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding="post", truncating="post")

# Train the model
model.fit(training_padded, train_df['label'], epochs=10, validation_split=0.2) # Increased epochs

def predict_message(pred_text):
    seq = tokenizer.texts_to_sequences([pred_text])  # Convert to numerical sequence
    padded_seq = pad_sequences(seq, maxlen=MAX_LENGTH)  # Ensure same input shape

    prediction_prob = model.predict(padded_seq)[0][0]  # Get probability score
    label = "spam" if prediction_prob > 0.5 else "ham"  # Apply threshold

    return [prediction_prob, label]


# Example usage
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)  # Expected output: [some small number close to 0, 'ham']

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
